In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [10]:
d = pd.read_csv('bond-strength-dataset V4 - bond-strength-dataset V4.csv')
d.head()

,No.,(mm),(Mpa),(Gpa),(mm).1,(mm).2,(mm).3,(kN)
0,1,100.0,40.80,230.0,0.11,50.0,75.0,5.80
1,2,100.0,40.80,230.0,0.11,50.0,150.0,9.20
2,3,100.0,44.93,230.0,0.11,50.0,300.0,11.95
3,4,100.0,44.31,230.0,0.22,50.0,65.0,9.55
4,5,100.0,44.31,230.0,0.22,50.0,150.0,16.25


In [11]:
d.dtypes

No.        object
(mm)      float64
(Mpa)     float64
(Gpa)     float64
(mm).1    float64
(mm).2    float64
(mm).3    float64
(kN)      float64
dtype: object

In [12]:
d.shape

(855, 8)

In [13]:
d.columns


Index(['No.', '(mm)', '(Mpa)', '(Gpa)', '(mm).1', '(mm).2', '(mm).3', '(kN)'], dtype='object')

In [14]:
x = d.drop(['(kN)','No.'], axis="columns")
y = d['(kN)']

In [15]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.23, random_state=25)

In [16]:
x_train.shape

(658, 6)

In [17]:
from sklearn.ensemble import AdaBoostRegressor
z = AdaBoostRegressor(n_estimators=100, random_state=10)

In [18]:
z.fit(x_train, y_train)

AdaBoostRegressor(n_estimators=100, random_state=10)

In [19]:
yp = z.predict(x_test)

In [20]:
res = pd.DataFrame({'Actual': y_test, 'Predicted':yp})
res

,Actual,Predicted
367,20.14,23.990382
509,6.80,15.563636
796,6.40,12.986176
443,20.90,15.450360
410,14.40,15.757303
...,...,...
219,14.95,15.249610
730,11.38,20.255217
469,52.60,44.934237
144,12.71,19.128929


In [21]:
z.score(x_test, y_test)

0.7359246007904938

In [22]:
from sklearn.metrics import r2_score as r2
print(r2(y_test,yp))


0.7359246007904938


In [40]:
from sklearn.ensemble import RandomForestRegressor
zr = RandomForestRegressor(n_estimators=100, random_state=10)

In [41]:
zr.fit(x_train, y_train)

RandomForestRegressor(random_state=10)

In [42]:
ypr = zr.predict(x_test)

In [43]:
resr = pd.DataFrame({'Actual': y_test, 'Predicted':ypr})
resr

,Actual,Predicted
367,20.14,24.110229
509,6.80,7.321000
796,6.40,7.129245
443,20.90,19.747380
410,14.40,15.360235
...,...,...
219,14.95,16.052658
730,11.38,11.737653
469,52.60,46.723030
144,12.71,12.723633


In [44]:
from sklearn.metrics import r2_score as r2
print(r2(y_test,ypr))

0.9333676770808548


In [45]:
from xgboost import XGBRegressor
zx = XGBRegressor(n_estimators=900, learning_rate=0.09)
zx.fit(x_train, y_train, 
       early_stopping_rounds=5,
       eval_set=[(x_test, y_test)],
       verbose=False)

c:\Users\Parthiv\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.09, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=900, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [46]:
ypx = zx.predict(x_test)

In [47]:
from sklearn.metrics import r2_score as r2
print(r2(y_test,ypx))

0.9357120998936793


In [48]:
import pickle as pk

pk.dump(zx, open('model.pkl', 'wb'))

In [49]:
model = pk.load(open('model.pkl', 'rb'))

In [50]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [51]:
d = {'1':'a', '2':'b', '3':'c'}
for i in d.items():
    print(i)

('1', 'a')
('2', 'b')
('3', 'c')


In [52]:
r2s = []
rmse = []
mae = []

for i in [yp, ypr, ypx]:
    r2s.append(r2_score(y_test,i))
    rmse.append(np.sqrt(mean_squared_error(y_test, i)))
    mae.append(mean_absolute_error(y_test, i))

In [54]:
scores = pd.DataFrame({'RootMeanSquaredError':rmse, 'MeanAbsoluteError':mae, 'R2Score':r2s}, index=['AdaBoost', 'RandomForest', 'XGBoost'])
scores

,RootMeanSquaredError,MeanAbsoluteError,R2Score
AdaBoost,4.991274,4.291363,0.735925
RandomForest,2.507205,1.609851,0.933368
XGBoost,2.462703,1.639573,0.935712
